## Objetivo
#### Prever se um passageiro foi transportado para uma dimensão alternativa durante a colisão da nave espacial Titanic

   - PassengerId- Um ID único para cada passageiro. Cada ID assume a forma gggg_pponde ggggindica um grupo com o qual o passageiro está viajando e ppé seu número dentro do grupo. As pessoas em um grupo geralmente são membros da família, mas nem sempre.
   - HomePlanet- O planeta de onde o passageiro partiu, normalmente seu planeta de residência permanente.
   - CryoSleep- Indica se o passageiro optou por ser colocado em animação suspensa durante a viagem. Os passageiros em sono criogênico estão confinados em suas cabines.
   - Cabin- O número da cabine onde o passageiro está hospedado. Toma a forma deck/num/side, onde sidepode ser tanto Ppara Bombordo como Spara Estibordo .
   - Destination- O planeta para o qual o passageiro irá desembarcar.
   - Age- A idade do passageiro.
   - VIP- Se o passageiro pagou pelo serviço VIP especial durante a viagem.
   - RoomService, FoodCourt, ShoppingMall, Spa, VRDeck- Valor que o passageiro faturou em cada uma das muitas comodidades de luxo da Nave Espacial Titanic .
   - Name- O nome e sobrenome do passageiro.
   - Transported- Se o passageiro foi transportado para outra dimensão. Este é o alvo, a coluna que você está tentando prever.

In [311]:
import numpy as np 
import pandas as pd 
import statsmodels.api as sm
import sweetviz as sv

from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.naive_bayes import GaussianNB 
from sklearn.neural_network import MLPClassifier

## Exploring Data

In [353]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
ytest = pd.read_csv('sample_submission.csv')
train

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [313]:
result = pd.merge(test,
                 ytest[['PassengerId', 'Transported']],
                 on='PassengerId', 
                 how='inner')

test = result.drop(['Transported'], axis=1)
ytest = result[['PassengerId', 'Transported']]

In [329]:
train

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Id,Group,Deck,Num,Side
0,Europa,0.0,TRAPPIST-1e,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,1.0,B,0.0,P
1,Earth,0.0,TRAPPIST-1e,24.0,0.0,109.0,9.0,25.0,549.0,44.0,1,2,1.0,F,0.0,S
2,Europa,0.0,TRAPPIST-1e,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,0,3,1.0,A,0.0,S
3,Europa,0.0,TRAPPIST-1e,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,0,3,2.0,A,0.0,S
4,Earth,0.0,TRAPPIST-1e,16.0,0.0,303.0,70.0,151.0,565.0,2.0,1,4,1.0,F,1.0,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,Europa,0.0,55 Cancri e,41.0,1.0,0.0,6819.0,0.0,1643.0,74.0,0,9276,1.0,A,98.0,P
8689,Earth,1.0,PSO J318.5-22,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0,9278,1.0,G,1499.0,S
8690,Earth,0.0,TRAPPIST-1e,26.0,0.0,0.0,0.0,1872.0,1.0,0.0,1,9279,1.0,G,1500.0,S
8691,Europa,0.0,55 Cancri e,32.0,0.0,0.0,1049.0,0.0,353.0,3235.0,0,9280,1.0,E,608.0,S


In [241]:
train.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,8514.000000,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000
mean,28.827930,224.687617,458.077203,173.729169,311.138778,304.854791
std,14.489021,666.717663,1611.489240,604.696458,1136.705535,1145.717189
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,47.000000,76.000000,27.000000,59.000000,46.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000


In [242]:
#count Null values
print(
        train['HomePlanet'].isna().sum(), train['CryoSleep'].isna().sum()
      , train['CryoSleep'].isna().sum()
      , train['Cabin'].isna().sum()
      , train['Age'].isna().sum()
      , train['VIP'].isna().sum(), train['RoomService'].isna().sum()
      , train['FoodCourt'].isna().sum()
      , train['ShoppingMall'].isna().sum()
      , train['Spa'].isna().sum()
      , train['Destination'].isna().sum()
     )

201 217 217 199 179 203 181 183 208 183 182


In [243]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [244]:
print(train.nunique())
train.dtypes

PassengerId     8693
HomePlanet         3
CryoSleep          2
Cabin           6560
Destination        3
Age               80
VIP                2
RoomService     1273
FoodCourt       1507
ShoppingMall    1115
Spa             1327
VRDeck          1306
Name            8473
Transported        2
dtype: int64


PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

# Transform Data

#### Replace and Split values

In [332]:
train['Transported'] = train['Transported'].replace([False, True],[0,1])
ytest['Transported'] = ytest['Transported'].replace([False, True],[0,1])
train['CryoSleep'] = train['CryoSleep'].replace([False, True],[0,1])
train['VIP'] = train['VIP'].replace([False, True],[0,1])
test['CryoSleep'] = test['CryoSleep'].replace([False, True],[0,1])
test['VIP'] = test['VIP'].replace([False, True],[0,1])


train[['Id','Group']] = train.PassengerId.str.split("_",expand=True,)
test[['Id','Group']] = test.PassengerId.str.split("_",expand=True,)
ytest[['Id','Group']]  = ytest.PassengerId.str.split("_",expand=True,)


train[['Deck','Num','Side']] = train.Cabin.str.split("/",expand=True,)
test[['Deck','Num','Side']] = test.Cabin.str.split("/",expand=True,)

train.drop(['Name', 'Cabin','PassengerId'], inplace=True, axis=1) #dropando colunas que não serão utilizadas
test.drop(['Name', 'Cabin','PassengerId'], inplace=True, axis=1)
ytest.drop(['PassengerId', 'Group'], inplace=True, axis=1)
ytest = ytest[['Id', 'Transported']]


In [333]:
ytest['Id'] =  ytest['Id'].astype(int)

test['Id'] =  test['Id'].astype(int)
test['Group'] =  test['Group'].astype(float)
test['Num'] =  test['Num'].astype(float)

train['Id'] =  train['Id'].astype(int)
train['Group'] =  train['Group'].astype(float)
train['Num'] =  train['Num'].astype(float)

train['CryoSleep'] =  train['CryoSleep'].astype(float)
test['CryoSleep'] =  test['CryoSleep'].astype(float)
train['VIP'] =  train['VIP'].astype(float)
test['VIP'] =  test['VIP'].astype(float)

In [338]:
train = train.query('Id <= 6000')
test = test.query('Id > 6000')
ytest = ytest.query('Id > 6000')

#### Remove Null Values

In [339]:
train['HomePlanet'] = train['HomePlanet'].fillna('other')
test['HomePlanet'] = test['HomePlanet'].fillna('other')

train['CryoSleep'] = train['CryoSleep'].fillna(test['CryoSleep'].mean())
test['CryoSleep'] = test['CryoSleep'].fillna(test['CryoSleep'].mean())

train['Destination'] = train['Destination'].fillna('other')
test['Destination'] = test['Destination'].fillna('other')

train['Age'] = train['Age'].fillna(train['Age'].mean())
test['Age'] = test['Age'].fillna(test['Age'].mean())

train['VIP'] = train['VIP'].fillna(test['VIP'].mean())
test['VIP'] = test['VIP'].fillna(test['VIP'].mean())

train['RoomService'] = train['RoomService'].fillna(test['RoomService'].mean())
test['RoomService'] = test['RoomService'].fillna(test['RoomService'].mean())

train['FoodCourt'] = train['FoodCourt'].fillna(test['FoodCourt'].mean())
test['FoodCourt'] = test['FoodCourt'].fillna(test['FoodCourt'].mean())

train['ShoppingMall'] = train['ShoppingMall'].fillna(test['ShoppingMall'].mean())
test['ShoppingMall'] = test['ShoppingMall'].fillna(test['ShoppingMall'].mean())  

train['Spa'] = train['Spa'].fillna(test['Spa'].mean())
test['Spa'] = test['Spa'].fillna(test['Spa'].mean()) 

train['VRDeck'] = train['VRDeck'].fillna(test['VRDeck'].mean())
test['VRDeck'] = test['VRDeck'].fillna(test['VRDeck'].mean()) 

train['Deck'] = train['Deck'].fillna('other')
test['Deck'] = test['Deck'].fillna('other')

train['Num'] = train['Num'].fillna(test['Num'].mean())
test['Num'] = test['Num'].fillna(test['Num'].mean())

train['Side'] = train['Side'].fillna('other')
test['Side'] = test['Side'].fillna('other')

C:\Users\narac\AppData\Local\Temp\ipykernel_11876\1304123068.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['HomePlanet'] = train['HomePlanet'].fillna('other')
C:\Users\narac\AppData\Local\Temp\ipykernel_11876\1304123068.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['HomePlanet'] = test['HomePlanet'].fillna('other')
C:\Users\narac\AppData\Local\Temp\ipykernel_11876\1304123068.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

C:\Users\narac\AppData\Local\Temp\ipykernel_11876\1304123068.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Deck'] = test['Deck'].fillna('other')
C:\Users\narac\AppData\Local\Temp\ipykernel_11876\1304123068.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Num'] = train['Num'].fillna(test['Num'].mean())
C:\Users\narac\AppData\Local\Temp\ipykernel_11876\1304123068.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [340]:
print(
        train['HomePlanet'].isna().sum(), 
    train['CryoSleep'].isna().sum()
      , train['CryoSleep'].isna().sum()
      , train['Age'].isna().sum()
      , train['VIP'].isna().sum(), train['RoomService'].isna().sum()
      , train['FoodCourt'].isna().sum()
      , train['ShoppingMall'].isna().sum()
      , train['Spa'].isna().sum()
      , train['Destination'].isna().sum()
     )

0 0 0 0 0 0 0 0 0 0


In [341]:
test.dtypes
test.groupby('CryoSleep').nunique()

,HomePlanet,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Id,Group,Deck,Num,Side
CryoSleep,,,,,,,,,,,,,,
0.000000,4,4,70,3,374,387,313,365,353,766,8,9,640,3
0.381015,4,4,25,3,19,12,16,20,17,36,7,8,36,3
1.000000,4,4,65,3,2,2,2,2,2,478,8,8,405,3


#### Analyzing the distribution of data

In [320]:
# my_report = sv.analyze(testv )
# my_report.show_html()

#### Categorize columns (dummies)

In [342]:
ytrain = train['Transported']
y_var = 'Transported'
xtrain = train.drop(['Transported'], axis=1)
xtest = test

In [343]:
xtrain = pd.get_dummies(xtrain,drop_first=True)
xtest = pd.get_dummies(xtest,drop_first=True)
xtest.head(3)

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Id,Group,...,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,Deck_other,Side_S,Side_other
2723,1.0,66.0,0.0,0.0,0.0,0.0,0.0,0.0,6002,1.0,...,0,0,0,1,0,0,0,0,0,0
2724,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6005,1.0,...,0,0,0,0,0,1,0,0,1,0
2725,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,6005,2.0,...,0,0,0,0,0,1,0,0,1,0


## Feature Engineering

In [344]:
ytrain.head(3)

0    0
1    1
2    0
Name: Transported, dtype: int64

# Models

In [345]:
##### deixou o ytrain e ytest igual, mas o xtrain tá diferente..
xtest.count()


CryoSleep                    1554
Age                          1554
VIP                          1554
RoomService                  1554
FoodCourt                    1554
ShoppingMall                 1554
Spa                          1554
VRDeck                       1554
Id                           1554
Group                        1554
Num                          1554
HomePlanet_Europa            1554
HomePlanet_Mars              1554
HomePlanet_other             1554
Destination_PSO J318.5-22    1554
Destination_TRAPPIST-1e      1554
Destination_other            1554
Deck_B                       1554
Deck_C                       1554
Deck_D                       1554
Deck_E                       1554
Deck_F                       1554
Deck_G                       1554
Deck_T                       1554
Deck_other                   1554
Side_S                       1554
Side_other                   1554
dtype: int64

### Decision Tree 

In [346]:
clf_dec = DecisionTreeClassifier(max_depth=2, min_samples_leaf=20, min_samples_split=0.2)
clf_dec.fit(xtrain, ytrain)
# print(clf_dec.score(xtest, ytest))
# print(f1_score(ytest, clf_dec.predict(xtest)))

DecisionTreeClassifier(max_depth=2, min_samples_leaf=20, min_samples_split=0.2)

In [347]:
print(clf_dec.score(xtest, ytest))

ValueError: Classification metrics can't handle a mix of multiclass-multioutput and binary targets

In [352]:
# xtest
# xtrain
ytest

,Id,Transported
2723,6002,0
2724,6005,0
2725,6005,0
2726,6005,0
2727,6005,0
...,...,...
4272,9266,0
4273,9269,0
4274,9271,0
4275,9273,0


In [258]:
ytrain

0       0
1       1
2       0
3       0
4       1
       ..
8688    0
8689    0
8690    1
8691    0
8692    1
Name: Transported, Length: 8693, dtype: int64

In [260]:
xtrain

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Id,Group,...,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,Deck_other,Side_S,Side_other
0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1.0,...,1,0,0,0,0,0,0,0,0,0
1,0.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,2,1.0,...,0,0,0,0,1,0,0,0,1,0
2,0.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,3,1.0,...,0,0,0,0,0,0,0,0,1,0
3,0.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,3,2.0,...,0,0,0,0,0,0,0,0,1,0
4,0.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,4,1.0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0.0,41.0,1.0,0.0,6819.0,0.0,1643.0,74.0,9276,1.0,...,0,0,0,0,0,0,0,0,0,0
8689,1.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,9278,1.0,...,0,0,0,0,0,1,0,0,1,0
8690,0.0,26.0,0.0,0.0,0.0,1872.0,1.0,0.0,9279,1.0,...,0,0,0,0,0,1,0,0,1,0
8691,0.0,32.0,0.0,0.0,1049.0,0.0,353.0,3235.0,9280,1.0,...,0,0,0,1,0,0,0,0,1,0


In [256]:
ytest.dtypes

Id             float64
Transported      int64
dtype: object

In [257]:
ytest

,Id,Transported
0,13.0,0
1,18.0,0
2,19.0,0
3,21.0,0
4,23.0,0
...,...,...
4272,9266.0,0
4273,9269.0,0
4274,9271.0,0
4275,9273.0,0


In [236]:
xtest

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Id,Group,...,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,Deck_other,Side_S,Side_other
0,1.0,27.000000,0.0,0.0,0.0,0.0,0.0,0.0,13.0,1.0,...,0,0,0,0,0,1,0,0,1,0
1,0.0,19.000000,0.0,0.0,9.0,0.0,2823.0,0.0,18.0,1.0,...,0,0,0,0,1,0,0,0,1,0
2,1.0,31.000000,0.0,0.0,0.0,0.0,0.0,0.0,19.0,1.0,...,0,1,0,0,0,0,0,0,1,0
3,0.0,38.000000,0.0,0.0,6652.0,0.0,181.0,585.0,21.0,1.0,...,0,1,0,0,0,0,0,0,1,0
4,0.0,20.000000,0.0,10.0,0.0,635.0,0.0,0.0,23.0,1.0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,1.0,34.000000,0.0,0.0,0.0,0.0,0.0,0.0,9266.0,2.0,...,0,0,0,0,0,1,0,0,1,0
4273,0.0,42.000000,0.0,0.0,847.0,17.0,10.0,144.0,9269.0,1.0,...,0,0,0,0,0,0,0,1,0,1
4274,1.0,28.658146,0.0,0.0,0.0,0.0,0.0,0.0,9271.0,1.0,...,0,0,1,0,0,0,0,0,0,0
4275,0.0,28.658146,0.0,0.0,2680.0,0.0,0.0,523.0,9273.0,1.0,...,0,0,1,0,0,0,0,0,0,0


- criar um arvore e entender quais são as features engen.
- remover id e num e ver o que isso impacta no modelo
- CORTAR OS INDICES DOS DADOS (separar metade metade)
